# AuxTel run 12/09/2023 - 15/09/2023 
# Focus estimation with cylindrical lens 

- SITCOM ticket: SITCOM-1012 

- Author: Martín Rodríguez Monroy 
- email: rodriguez-monroy@ijclab.in2p3.fr \ martin.rodriguez.monroy@gmail.com 
- Affiliation: IJCLab - CNRS 
- Creation date: 13/09/2023 
- Last update: 


In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.dates as mdates

import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch


from astropy.io import fits




In [ ]:
# Assembly task
# https://github.com/lsst/ip_isr/blob/main/python/lsst/ip/isr/isrTask.py

from lsst.ip.isr.assembleCcdTask import (AssembleCcdConfig, AssembleCcdTask)
from lsst.ip.isr.isrTask import (IsrTask, IsrTaskConfig)

#https://github.com/lsst/ip_isr/blob/main/python/lsst/ip/isr/overscan.py
from lsst.ip.isr import  OverscanCorrectionTaskConfig, OverscanCorrectionTask

In [ ]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Butler 

In [ ]:
import lsst.daf.butler as dafButler

In [ ]:
embargo = True

In [ ]:
if embargo:
    repo="/sdf/group/rubin/repo/oga/"
else:
    repo = "/sdf/group/rubin/repo/main"
butler = dafButler.Butler(repo)
registry = butler.registry

In [ ]:
cameraName = 'LATISS'
calibCollections = ['LATISS/defaults','LATISS/raw/all']
for col in registry.queryCollections("*LATISS/calib*"):
    #print(col)
    calibCollections.append(col)

# Functions 

In [ ]:
def quad(x,a,b,c):
    return a*x**2.+b*x+c

# Paths 

In [ ]:
outdir_base = 'output_fits/'
if os.path.exists(outdir_base)==False:
    os.mkdir(outdir_base)
outdir_label = 'cyl_lens_flat_{0}/'

We check the different dimension records existing for *physical_filter*: 

In [ ]:
phys_filters = list(registry.queryDimensionRecords('physical_filter',where="instrument='LATISS'"))

Print physical_filters with hologram in place: 

In [ ]:
for f_ in phys_filters:
    if 'holo' in f_.name:
        print(f_.name)

In [ ]:
obs_type = 'engtest'
obs_day = 20230913
physical_filter = 'cyl_lens~holo4_003' #Note that there are no exposure with this config for the moment 

In [ ]:
df_science = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num','time_start',
                                    'time_end' ,'type', 'target','filter_disp','zenith_angle',
                                    'expos','ra','dec','skyangle','science_program'])

In [ ]:

where_exps = "instrument='LATISS' AND exposure.observation_type='{0}' AND exposure.day_obs={1} AND physical_filter='{2}'".format(obs_type,obs_day,physical_filter)
#where_exps = "instrument='LATISS' AND exposure.observation_type='{0}' AND exposure.day_obs={1}".format(obs_type,obs_day)
for i, info in enumerate(registry.queryDimensionRecords('exposure',where=where_exps)):
    
    #if info.observation_type=='science':
    id_ = info.id
    obs_id_ = info.obs_id
    day_obs_ = info.day_obs
    seq_num_ = info.seq_num
    timespan_ = info.timespan
    timespan_begin_ = pd.to_datetime(timespan_.begin.to_string())
    timespan_end_ = pd.to_datetime(timespan_.end.to_string())
    timespan_begin_jd_ = timespan_.begin.jd
    timespan_begin_mjd_ = timespan_.begin.mjd

    observation_type_ = info.observation_type
    target_name_ = info.target_name
    physical_filter_ = info.physical_filter
    zenith_angle_ = info.zenith_angle
    exposure_time_ = info.exposure_time
    tracking_ra_ = info.tracking_ra
    tracking_dec_ = info.tracking_dec
    sky_angle_ = info.sky_angle
    science_program_ = info.science_program

    df_science.loc[i] = [id_, obs_id_, day_obs_, seq_num_,timespan_begin_,timespan_end_ ,observation_type_, \
                         target_name_, physical_filter_, zenith_angle_, exposure_time_,tracking_ra_, \
                         tracking_dec_, sky_angle_, science_program_]

    if i < 2:
        print(i)
        print(info)
        print("\t timespan:            ",info.timespan)
        print("\t timespan.begin:      ",info.timespan.begin)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
        print("-----------------------------------------------------")
    

In [ ]:
print('Number of entries = ',len(df_science))

In [ ]:
df_science

Select exposures: 

In [ ]:
seq_nums = np.sort(np.array(df_science.seq_num))
print(seq_nums)

In [ ]:
ids = np.sort(np.array(df_science.id))
print(ids)

We test retrieving one of them: 

In [ ]:
seq_num0 = seq_nums[0]
exp_id0 = ids[0]
print(exp_id0)

In [ ]:
test_img = butler.get('raw', dataId={'exposure': exp_id0, 'instrument': 'LATISS', 'detector': 0}, collections = calibCollections)

In [ ]:
fig = plt.figure(figsize=(12,10))
afw_display = afwDisplay.Display(frame=fig)
afw_display.scale('linear', 'zscale',None)
afw_display.mtv(test_img.image)

We do the ISR on this image the same way we do it with the spectra images (i.e., we do our pseudo-postISRCCD): 

In [ ]:
# configuration
isr_config =  IsrTaskConfig()

In [ ]:
isr_config.doDark = False
isr_config.doFlat =  False
isr_config.doFringe = False
isr_config.doDefect = True
isr_config.doLinearize = False
isr_config.doCrosstalk =  False
isr_config.doSaturationInterpolation = False
isr_config.overscan.fitType: 'MEDIAN_PER_ROW'
isr_config.doBias: True


In [ ]:
isr_task = IsrTask(config=isr_config)

In [ ]:
butler = dafButler.Butler(repo, collections=calibCollections)
camera = butler.get('camera', instrument=cameraName)
#bias = butler.get('bias',instrument=cameraName,detector=0)
#defects = butler.get('defects',instrument=cameraName,detector=0)

In [ ]:

bias = butler.get("bias",instrument=cameraName, exposure= exp_id0, detector=0, collections=calibCollections)
defects = butler.get('defects',instrument=cameraName, exposure= exp_id0,detector=0,collections=calibCollections)

#fast ISR 
isr_test = isr_task.run(test_img,bias=bias,defects=defects)


In [ ]:
isr_test.exposure.visitInfo.getFocusZ()

In [ ]:
isr_test.exposure.visitInfo.getId()

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
im = ax.imshow(isr_test.exposure.image.array,cmap="gray",origin='lower',norm=LogNorm())
fig.colorbar(im,ax=ax)

# Create sum exposure from individual exposures 

## 1. Do pseudo-postISRCCD 

In [ ]:
print(seq_nums)

In [ ]:
exp_dict = {}
ampli_boxes = {}
ampli_names = []
focus_dict = {}
for i,exp_id_ in enumerate(ids):
    print(exp_id_)
    raw_img_ = butler.get('raw', dataId={'exposure': exp_id_, 'instrument': 'LATISS', 'detector': 0}, collections = calibCollections)
    print('Raw image retrieved')
    
    bias_ = butler.get("bias",instrument=cameraName, exposure= exp_id_, detector=0, collections=calibCollections)
    defects_ = butler.get('defects',instrument=cameraName, exposure= exp_id_,detector=0,collections=calibCollections)
    
    #fast ISR 
    print('Running pseudo-ISR')
    isr_img_ = isr_task.run(raw_img_,bias=bias_,defects=defects_)
    
    assert (isr_img_.exposure.visitInfo.getId()==exp_id_)
    focus_ = isr_img_.exposure.visitInfo.getFocusZ()
    focus_dict[exp_id_] = focus_
    
    exp_dict[exp_id_] = isr_img_
    print('-----------------')
    
    if i==0:
        for ampIdx, amp in enumerate(raw_img_.getDetector()):
            ampli_name_ = amp.getName()
            ampli_names.append(ampli_name_)
            xbegin = amp.getBBox().x.begin
            xend = amp.getBBox().x.end
            ybegin = amp.getBBox().y.begin
            yend = amp.getBBox().y.end
            ampli_boxes[ampli_name_] = (xbegin,xend,ybegin,yend)
            
            md = raw_img_.getMetadata().toDict()
    

In [ ]:
ampli_boxes

In [ ]:
mapampid = [0,1,2,3,4,5,6,7,15,14,13,12,11,10,9,8]

In [ ]:
num_amplis = len(mapampid)

In [ ]:
fig = plt.figure(figsize=(20,60))
#for i,exp_ in enumerate(exp_list):
for i,expi in enumerate(ids):
    ax = fig.add_subplot(len(ids),1,i+1)
    exp_ = exp_dict[expi]
    im = ax.imshow(exp_.exposure.image.array,cmap="gray",origin='lower',norm=LogNorm())
    fig.colorbar(im,ax=ax)

In [ ]:
exp0 = exp_dict[ids[0]].exposure.image.array

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111)
im = ax.imshow(exp0,cmap="gray",origin='lower',vmax=1000)#,norm=LogNorm())
fig.colorbar(im,ax=ax)

In [ ]:
pos = 1750
delta_pos = 5
pmin = 200
pmax = 400
pix = np.arange(pmin,pmax)

In [ ]:
fig = plt.figure(figsize=(8,40))
for i,expi in enumerate(ids):
    ax = fig.add_subplot(len(ids),1,i+1)
    exp_ = exp_dict[expi].exposure.image.array
    dd_ = exp_[pmin:pmax,pos]
    focus_ = focus_dict[expi]
    ax.plot(pix,dd_,label='{0}, {1:.4f}'.format(expi,focus_))
    ax.grid()
    ax.set_ylim(0,60000)
    ax.legend()

In [ ]:
max_dict = {}
sigma_dict = {}
fig = plt.figure(figsize=(8,50))
for i,expi in enumerate(ids):
    ax = fig.add_subplot(len(ids),1,i+1)
    exp_ = exp_dict[expi].exposure.image.array
    dd_ = exp_[pmin:pmax,pos-delta_pos:pos+delta_pos]
    focus_ = focus_dict[expi]
    dd_sum = np.sum(dd_,axis=1)
    
    max_ = np.max(dd_sum)
    xmax_ = pix[np.where(dd_sum==max_)[0]][0]
    mu_ = np.sum(pix*dd_sum)/np.sum(dd_sum)
    sigma_ = np.sqrt(np.sum(dd_sum*(pix-mu_)**2.)/np.sum(dd_sum))
    
    max_dict[expi] = max_
    sigma_dict[expi] = sigma_
    
    ax.plot(pix,dd_sum,label='{0}, {1:.4f}'.format(id_,focus_))
    ax.axvline(x=xmax_,ls='--',color='r',label='Max = {0}'.format(xmax_))
    ax.axvline(x=mu_,ls='--',color='b',label=r'$\mu = $'+'{0:.2f}'.format(mu_))
    ax.axvline(x=mu_-sigma_,ls='--',color='orange',label=r'$\sigma = $'+'{0:.2f}'.format(sigma_))
    ax.axvline(x=mu_+sigma_,ls='--',color='orange')
    ax.set_ylim(0,600000)
    ax.grid()
    ax.set_xlabel('Pixel')
    ax.set_ylabel('Sum ADU')
    ax.legend()
plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111)
focus_list = []
sigma_list = []
for expi in ids:
    ax.scatter(focus_dict[expi],sigma_dict[expi],marker='o',color='orange')
    focus_list.append(focus_dict[expi])
    sigma_list.append(sigma_dict[expi])
focus_list = np.array(focus_list)
sigma_list = np.array(sigma_list)

ax.plot([],[],ls='',marker='o',color='orange',label='Data')

psigma,covsigma = curve_fit(quad,focus_list,sigma_list)

sigma_fit = quad(focus_list,psigma[0],psigma[1],psigma[2])

ax.plot(focus_list,sigma_fit,ls='-',color='orange',label='Fit')

foptdata = focus_list[np.where(sigma_list==np.min(sigma_list))[0]][0]
foptfit = focus_list[np.where(sigma_fit==np.min(sigma_fit))[0]][0]

print('Optimal focus from data = ', foptdata)
print('Optimal focus from fit = ', foptfit)

ax.axvline(x=foptdata,ls='--',color='g',label=r'$F_{opti}^{data} = $'+'{0:.2f}'.format(foptdata))
ax.axvline(x=foptfit,ls='--',color='magenta',label=r'$F_{opti}^{fit} = $'+'{0:.2f}'.format(foptfit))

ax.grid()
ax.set_xlabel('Focus')
ax.set_ylabel(r'$\sigma$ [pixel]')
ax.legend(loc="best")


In [ ]:
fig = plt.figure(figsize=(10,8))
ax1 = fig.add_subplot(111)
ax2 = ax1.twinx()
focus_list = []
max_list = []
sigma_list = []
for expi in ids:
    ax1.scatter(focus_dict[expi],max_dict[expi],marker='o',color='b')
    ax2.scatter(focus_dict[expi],sigma_dict[expi],marker='s',color='orange')
    focus_list.append(focus_dict[expi])
    max_list.append(max_dict[expi])
    sigma_list.append(sigma_dict[expi])
focus_list = np.array(focus_list)
max_list = np.array(max_list)
sigma_list = np.array(sigma_list)

pmax,covmax = curve_fit(quad,focus_list,max_list)
psigma,covsigma = curve_fit(quad,focus_list,sigma_list)

max_fit = quad(focus_list,pmax[0],pmax[1],pmax[2])
sigma_fit = quad(focus_list,psigma[0],psigma[1],psigma[2])

print(focus_list[np.where(sigma_fit==np.min(sigma_fit))[0]])

ax1.plot(focus_list,max_fit,ls='-',color='b')
ax2.plot(focus_list,sigma_fit,ls='-',color='orange')

fmax = focus_list[np.where(max_list==np.max(max_list))[0]][0]
fsigma = focus_list[np.where(sigma_list==np.min(sigma_list))[0]][0]

print(fmax)
print(fsigma)

ax1.axvline(x=fmax,ls='--',color='b',alpha=0.7)
ax2.axvline(x=fsigma,ls='--',color='orange',alpha=0.7)

ax1.axvspan(fmax,fsigma,color='yellow',alpha=0.3)

ax1.grid()
ax1.set_xlabel('Focus')
ax1.set_ylabel('Max')
ax2.set_ylabel(r'$\sigma$ [pixel]')
